# Final

In this notebook we run the configurations we have selected based on the results of our `experiment` notebook.
The main difference here is that we now attach the an `IOH` logger to the problem object so we can export and visualize the results of these final runs in IOanalyzer.

Imports

In [ ]:
import os
from itertools import product

from nasbench.api import NASBench

from main import init_ipynb
from utils import get_directories, ProgressBar

In [ ]:
DIRS = get_directories(os.path.join(os.path.abspath(''), 'final.ipynb'))

In [ ]:
NB = NASBench(DIRS['data'] + 'nasbench_only108.tfrecord', seed=42)

---

## Genetic Algorithm

In [ ]:
exp_id = 'GA_fin'
DIRS['csv_exp'] = DIRS['csv'] + exp_id + os.sep
if not os.path.exists(DIRS['csv_exp']):
    os.mkdir(DIRS['csv_exp'])

for n, p in DIRS.items():
    print(f'{n}: {p}')

In [ ]:
default_args = dict(budget=5000, verbose=0, repetitions=20, log=True, seed=42)

In [ ]:
mutations = [('u', 0.1), ('b', 2)]
recombinations = [('kp', 1), ('u', None)]
pop_divisions = [(100, 20, 100), (100, 20, 80), (100, 10, 100), (100, 10, 90)]

# create a list of all possible combinations of the above parameters
combinations = list(product(mutations, recombinations, pop_divisions))
n_combs = len(combinations)
progress = ProgressBar(n_combs, exp_id)

for i, (mut, rec, (ps, mu, lm)) in enumerate(combinations):
    args = default_args.copy()
    args['optimizer'] = 'GA'
    args['selection'] = 'ts'
    args['mutation'] = mut[0]
    if mut[0] == 'u':
        args['mut_r'] = mut[1]
        args['mut_b'] = 1
    else:
        args['mut_b'] = mut[1]
        args['mut_r'] = None
    args['recombination'] = rec[0]
    args['xp'] = rec[1]
    args['population_size'] = ps
    args['mu_'] = mu
    args['lambda_'] = lm
    run_id = f'GA_{ps}_{mu}_{lm}_{mut[0]}({mut[1]})_{rec[0]}({rec[1]})'.replace('(None)', '')
    args['run_id'] = run_id

    # run the experiment
    init_ipynb(NB, args, save_to=DIRS['csv_exp'])

    progress(i)

---

## Evolution Strategy

In [ ]:
exp_id = 'ES_fin'
DIRS['csv_exp'] = DIRS['csv'] + exp_id + os.sep
if not os.path.exists(DIRS['csv_exp']):
    os.mkdir(DIRS['csv_exp'])

for n, p in DIRS.items():
    print(f'{n}: {p}')

In [ ]:
recombinations = ['dg', 'i']
sigmas = [0.1, 0.5]
pop_divisions = [(100, 20, 100), (100, 20, 80), (100, 10, 100), (100, 10, 90)]

# create a list of all possible combinations of the above parameters
combinations = list(product(recombinations, sigmas, pop_divisions))
n_combs = len(combinations)
progress = ProgressBar(n_combs, exp_id)

for i, (rec, sigma, (ps, mu, lm)) in enumerate(combinations):
    args = default_args.copy()
    args['optimizer'] = 'ES'
    args['recombination'] = rec
    args['tau_'] = 0.5
    args['sigma_'] = sigma
    args['chunk_size'] = 3
    args['individual_sigmas'] = True
    args['population_size'] = ps
    args['mu_'] = mu
    args['lambda_'] = lm
    run_id = f'ES_{ps}_{mu}_{lm}_{rec}_{sigma}'
    args['run_id'] = run_id

    # run the experiment
    init_ipynb(NB, args, save_to=DIRS['csv_exp'])

    progress(i)